In [1]:
import pandas as pd
import numpy as np
import json
import gzip
import seaborn as sns
from datetime import datetime
from pandas.io.json import json_normalize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
%matplotlib inline

In [2]:
def open_json(data_file):    
    with open(data_file, 'r') as f:
        json_records = []
        for line in f:
            record = json.loads(line)
            json_records.append(record)
    return json_records

def gender_to_bin(g):
    if g == 'M':
        return 1
    else:
        return 0

def bin_to_gender(g):
    if g == 1:
        return 'M'
    else:
        return 'F'

def to_weekday(timestamp):
    try:
        weekday = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S").weekday()
    except ValueError:
        weekday = datetime.strptime(timestamp, "%Y-%m-%d").weekday()
    except TypeError:
        weekday = 1
    return weekday

## Create 5 DataFrames: catalog, purchase, pageview, target_purchase, target_pageview

In [87]:
%%time
catalog = pd.read_csv('data/catalog.gz', usecols=['pid', 'current_price', 'category', 'sub_category', 'sub_sub_category'])
for i,val in enumerate(catalog.current_price.values):
    if np.isnan(val):
        catalog.current_price.values[i] = 0

purchase_data = open_json('data/purchase_data')           
purchase = json_normalize(purchase_data, 'products', ['date','gender','uid'])

pageview_data = open_json('data/products_data')   
pageview = pd.DataFrame(pageview_data, columns=['productId', 'timestamp', 'gender', 'uid'])
pageview.columns = ['pid', 'date', 'gender', 'uid']


purchase_data = open_json('data/purchase_target')           
target_purchase = json_normalize(purchase_data, 'products', ['date','uid'])

pageview_data = open_json('data/products_target')   
target_pageview = pd.DataFrame(pageview_data, columns=['productId', 'timestamp', 'uid'])
target_pageview.columns = ['pid', 'date', 'uid']

CPU times: user 9.62 s, sys: 76 ms, total: 9.7 s
Wall time: 10 s


## Transform gender and date into numbers

In [88]:
%%time
purchase.gender = list(map(gender_to_bin, purchase.gender.values))
purchase.date = list(map(to_weekday, purchase.date.values))

pageview.gender = list(map(gender_to_bin, pageview.gender.values))
pageview.date = list(map(to_weekday, pageview.date.values))


target_purchase.date = list(map(to_weekday, target_purchase.date.values))
target_pageview.date = list(map(to_weekday, target_pageview.date.values))

CPU times: user 21.1 s, sys: 20 ms, total: 21.2 s
Wall time: 21.1 s


## Set dummies vars for categorical features

In [89]:
%%time
categorical = ['category', 'sub_category', 'sub_sub_category', 'date']

purchase = purchase.join(catalog.set_index('pid'), on='pid')
purchase = pd.get_dummies(purchase, columns=categorical).iloc[:,1:]

pageview = pageview.join(catalog.set_index('pid'), on='pid')
pageview = pd.get_dummies(pageview, columns=categorical).iloc[:,1:]

target_purchase = target_purchase.join(catalog.set_index('pid'), on='pid')
target_purchase = pd.get_dummies(target_purchase, columns=categorical).iloc[:,1:]

target_pageview = target_pageview.join(catalog.set_index('pid'), on='pid')
target_pageview = pd.get_dummies(target_pageview, columns=categorical).iloc[:,1:]

CPU times: user 1.32 s, sys: 188 ms, total: 1.5 s
Wall time: 1.5 s


## Merge users with same uid (sum entries) and join DFs

In [91]:
len(target_purchase.uid.values)

12681

In [78]:
%%time
#quantity = purchase.pop('quantity')
purchase = purchase.groupby(['uid','gender'], as_index=False).sum()
pageview = pageview.groupby(['uid','gender'], as_index=False).sum()
data = purchase.join(pageview.set_index(['uid','gender']), on=['uid','gender'], rsuffix='_v')


target_purchase = target_purchase.groupby(['uid'], as_index=False).sum()
target_pageview = target_pageview.groupby(['uid'], as_index=False).sum()
target_data = target_purchase.join(target_pageview.set_index('uid'), on='uid', rsuffix='_v')

CPU times: user 1.68 s, sys: 740 ms, total: 2.42 s
Wall time: 2.61 s


In [79]:
data = data.fillna(value=0)
target_data = target_data.fillna(value=0)

In [80]:
features = data.columns[3:]
target = data.columns[1]
features2 = target_data.columns[2:]
features = list(set(features)&set(features2))

In [81]:
X = data[features]
Y = data[target]
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=42)

In [82]:
%%time
forest = RandomForestClassifier(n_estimators=100)
forest.fit(x_train, y_train)
y_pred = forest.predict(x_test)
score = metrics.accuracy_score(y_test, y_pred)

CPU times: user 2.93 s, sys: 8 ms, total: 2.94 s
Wall time: 2.92 s


In [83]:
score

0.78935427574171024

In [84]:
X = target_data[features]
answer = forest.predict(X)
users = target_data.uid.values
ans = []
for i,u in enumerate(users):
    if answer[i]:
        g = 'M'
    else:
        g = 'F'
    obj = {'a':u, 'b':g}
    ans.append(obj)
    
import csv

with open('ans4.csv', 'wb') as f:
    w = csv.DictWriter(f, fieldnames=['a','b'])
    for obj in ans:
        w.writerow(obj)

In [117]:
df = pd.read_csv('ans79.csv',header=None)
df.columns = ['uid','g']

In [118]:
mod_df = df.groupby('uid').agg(lambda x:x.value_counts().index[0])

In [122]:
for i,u in enumerate(df.uid.values):
    df.g[i] = mod_df.loc[u].g

In [125]:
df.head()

,uid,g
0,b759ae76feb4521141c3c16eb7ba573b710b343d,M
1,0103b1b008902ebbac19c6a4b64670ab01f586d8,F
2,bd5417b2a012cd96a2be865dd1f69ead536b600c,F
3,3e8c80742066eac5e866bd4c1f7001791d018d72,M
4,74642552d7e2f326828862753fef15d6f7f3540a,F


In [128]:
df.to_csv('real_ans.csv',header=False,index=False)